# Геолокация сообщений

#### Импорт библиотек

In [1]:
import warnings

warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)

import os
import sys
import pandas as pd

folder = os.getcwd().split("\\examples")[0]
sys.path.append(folder)

from sloyka.src.utils.data_getter.data_getter import VKParser
from sloyka.src.geocoder.geocoder import Geocoder
from sloyka.src.geocoder.street_getter import Streets

INFO: Pandarallel will run on -1 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\trolo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\trolo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


2024-06-10 00:05:04,302 SequenceTagger predicts: Dictionary with 7 tags: O, S-Service, B-Service, E-Service, I-Service, <START>, <STOP>


In [2]:
df=pd.read_csv('F:/hui.csv')

In [3]:
result = Geocoder(osm_id = 7373058).run(osm_id = 7373058, tags = {"place": ["city", "suburb"], "admin_level": ["9"]}, date = "2024-04-22T00:00:00Z", df=df, text_column='text', group_column='group_name')

2024-06-10 00:05:10,006 SequenceTagger predicts: Dictionary with 5 tags: O, S-ADDRESS, B-ADDRESS, E-ADDRESS, I-ADDRESS


Processing category place:   0%|          | 0/2 [00:00<?, ?it/s]c:\Users\trolo\python3.10.9\envs\nirmapy\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
c:\Users\trolo\python3.10.9\envs\nirmapy\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
c:\Users\trolo\python3.10.9\envs\nirmapy\lib\site-packages\pandas\core\algorithms.py:522: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs

06-10 00:05 | INFO     | get_street started
06-10 00:05 | INFO     | extract_ner_street started


100%|██████████| 133/133 [00:00<00:00, 153.89it/s]

06-10 00:05 | INFO     | pattern1.sub started


...


  0%|          | 0/60 [00:00<?, ?it/s]c:\Users\trolo\python3.10.9\envs\nirmapy\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
100%|██████████| 60/60 [00:00<00:00, 769.23it/s]

06-10 00:05 | INFO     | get_stem started



c:\Users\trolo\python3.10.9\envs\nirmapy\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
c:\Users\trolo\python3.10.9\envs\nirmapy\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])


06-10 00:05 | INFO     | create_gdf started


100%|██████████| 23/23 [00:09<00:00,  2.48it/s]
c:\Users\trolo\python3.10.9\envs\nirmapy\lib\site-packages\pandas\core\algorithms.py:522: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  common = np.find_common_type([values.dtype, comps_array.dtype], [])


In [4]:
result

,Unnamed: 0,date,id,text,views.count,likes.count,reposts.count,type,link,from_id,...,initial_street,Toponims,full_street_name,location_options,addr_to_geocode,only_full_street_name,Location,geometry,other_geo_obj,geo_obj_tag
0,1,2024-06-09 14:20:00,176668,«Работа России. Знание — сила»: 40 школ и детс...,1745.0,2,0.0,post,NaN,NaN,...,Конгрессной,улица,Конгрессная улица 153 Краснодар Россия,['Конгрессная улица 153 Краснодар Россия'],Конгрессная улица 153 Краснодар Россия,Конгрессная улица,"Конгрессная улица, Прикубанский округ, Краснод...",POINT (38.97375 45.10411),NaN,street
1,2,2024-06-09 10:50:00,176663,Служба по контракту: социальные льготы и гаран...,5096.0,3,0.0,post,NaN,NaN,...,Ярославская,улица,Ярославская улица 8 Краснодар Россия,['Ярославская улица 8 Краснодар Россия'],Ярославская улица 8 Краснодар Россия,Ярославская улица,"8, Ярославская улица, 9-й км, Прикубанский окр...",POINT (38.98319 45.07689),NaN,street
2,4,2024-06-08 10:20:00,176650,🍒На летних ярмарках выходного дня — богатый ур...,2125.0,9,0.0,post,NaN,NaN,...,Архитектора Ишунина,None,улица Архитектора Ишунина Краснодар Россия,['улица Архитектора Ишунина Краснодар Россия'],улица Архитектора Ишунина Краснодар Россия,улица Архитектора Ишунина,"улица Архитектора Ишунина, Фестивальный, Прику...",POINT (38.96025 45.06853),NaN,street
3,13,2024-06-07 08:56:12,176598,📽️ В микрорайоне Гидростроителей пройдёт киноп...,5722.0,12,2.0,post,NaN,NaN,...,Гидростроителей,None,улица Гидростроителей Краснодар Россия,['улица Гидростроителей Краснодар Россия'],улица Гидростроителей Краснодар Россия,улица Гидростроителей,"улица Гидростроителей, Карасунский округ, Крас...",POINT (39.08189 45.00334),NaN,street
4,20,2024-06-06 10:27:31,176578,🚍 В Краснодаре изменится схема движения автобу...,6139.0,5,1.0,post,NaN,NaN,...,Тургенева,улица,улица Тургенева Краснодар Россия,['улица Тургенева Краснодар Россия'],улица Тургенева Краснодар Россия,улица Тургенева,"улица Тургенева, Прикубанский округ, Краснодар...",POINT (38.96106 45.06285),NaN,street
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,128,2024-06-03 03:25:29,176498,"А мне вот не мешают, чем брать гнить втридорог...",NaN,2,NaN,comment,NaN,365234283.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
136,129,2024-06-03 04:38:01,176500,Собаки торговых сетей вышли на охоту,NaN,3,NaN,comment,NaN,596232317.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
137,130,2024-06-04 11:04:38,176534,В панораме с..ть хотели и на участкового и на...,NaN,0,NaN,comment,NaN,279903238.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
138,131,2024-06-04 11:52:55,176536,"Татьяна, Проблема в барыгах! Где появляется ...",NaN,1,NaN,comment,NaN,279903238.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN


In [1]:
import warnings

warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)

import os
import sys
import pandas as pd

folder = os.getcwd().split("\\examples")[0]
sys.path.append(folder)
from sloyka.src.geocoder.city_objects_getter import OtherGeoObjects

INFO: Pandarallel will run on -1 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\trolo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\trolo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


2024-06-10 00:00:29,782 SequenceTagger predicts: Dictionary with 7 tags: O, S-Service, B-Service, E-Service, I-Service, <START>, <STOP>


In [ ]:
geobj = OtherGeoObjects().run(osm_id=7373058, df=df, text_column='text')

In [2]:
streets_df = Streets.run(7373058)

In [3]:
streets_df

,street,toponim_name,street_name
0,Эльбрусская улица,улица,эльбрусская
1,6-я Дорожная улица,улица,6-я дорожная
2,Лесная улица,улица,лесная
3,Палаточная улица,улица,палаточная
4,Горячеключевская улица,улица,горячеключевская
...,...,...,...
1611,1-я Трудовая улица,улица,1-я трудовая
1612,3-й Лучистый проезд,проезд,3-й лучистый
1613,Коллективная улица,улица,коллективная
1614,улица Шишкина,улица,шишкина


#### Загрузка данных

In [3]:
comments_parser = VKParser()

domain = 'krdru' #доменное имя группы
accsess_token='96cbbc1496cbbc1496cbbc14b795dfa8b8996cb96cbbc14f2a8294fa4c4c6fc7e753a93'


df = VKParser().run_parser(domain, accsess_token, step=100, cutoff_date='2024-03-20', number_of_messages=50)

2024-06-02 09:51:04


100%|██████████| 49/49 [00:38<00:00,  1.29it/s]

comments downloaded


In [ ]:
df

In [2]:
df=pd.read_csv('F:/hui.csv')

In [3]:
geobj = OtherGeoObjects().run(osm_id=7373058, df=df, text_column='text')

c:\Users\trolo\python3.10.9\envs\nirmapy\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
c:\Users\trolo\python3.10.9\envs\nirmapy\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
c:\Users\trolo\python3.10.9\envs\nirmapy\lib\site-packages\pandas\core\algorithms.py:522: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (

In [ ]:
geobj

In [7]:
import warnings

warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)

import os
import sys
import pandas as pd

folder = os.getcwd().split("\\examples")[0]
sys.path.append(folder)

from sloyka.src.utils.data_getter.data_getter import GeoDataGetter

INFO: Pandarallel will run on -1 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\trolo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\trolo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


2024-06-09 23:52:54,863 SequenceTagger predicts: Dictionary with 7 tags: O, S-Service, B-Service, E-Service, I-Service, <START>, <STOP>


In [11]:
hel_me = GeoDataGetter().get_features_from_id(osm_id=7373058, tags={"place": ["city"]}, selected_columns=['name', 'geometry'])

Processing category place:   0%|          | 0/1 [00:00<?, ?it/s]c:\Users\trolo\python3.10.9\envs\nirmapy\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
c:\Users\trolo\python3.10.9\envs\nirmapy\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
c:\Users\trolo\python3.10.9\envs\nirmapy\lib\site-packages\pandas\core\algorithms.py:522: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs

In [12]:
hel_me

,name,geometry
0,Краснодар,POINT (38.97724 45.03515)
1,Краснодар,"POLYGON ((38.79530 45.08267, 38.80402 45.09163..."


In [ ]:
geobj

In [1]:
import pandas as pd
import geopandas as gpd
import osmnx as ox

osm_id = 7373058

In [6]:
ox.geocode?

Signature: ox.geocode(query)
Docstring:
Geocode place names or addresses to (lat, lng) with the Nominatim API.

This geocodes the query via the Nominatim "search" endpoint.

Parameters
----------
query : string
    the query string to geocode

Returns
-------
point : tuple
    the (lat, lng) coordinates returned by the geocoder
File:      c:\users\trolo\python3.10.9\envs\nirmapy\lib\site-packages\osmnx\geocoder.py
Type:      function

In [4]:
city_boundary_gdf = ox.geocode_to_gdf("R" + str(osm_id), by_osmid=True)


In [5]:
display(city_boundary_gdf)

,geometry,bbox_north,bbox_south,bbox_east,bbox_west,place_id,osm_type,osm_id,lat,lon,class,type,place_rank,importance,addresstype,name,display_name
0,"POLYGON ((38.77109 45.05861, 38.77304 45.05794...",45.154006,44.967958,39.186319,38.771093,371598411,relation,7373058,45.035153,38.97724,place,city,16,0.597438,city,Krasnodar,"Krasnodar, Krasnodar Municipality, Krasnodar K..."


In [ ]:
geobj = OtherGeoObjects().run(osm_id=7373058, df=df, text_column='text')

In [3]:
df

,date,id,text,views.count,likes.count,reposts.count,type,link,from_id,post_id,parents_stack,group_name
0,2024-06-09 17:05:00,176673,В Краснодаре состоялся краевой смотр творчески...,3464.0,2,1.0,post,NaN,NaN,NaN,NaN,Город Краснодар
1,2024-06-09 14:20:00,176668,«Работа России. Знание — сила»: 40 школ и детс...,1664.0,2,0.0,post,NaN,NaN,NaN,NaN,Город Краснодар
2,2024-06-09 10:50:00,176663,Служба по контракту: социальные льготы и гаран...,4921.0,3,0.0,post,NaN,NaN,NaN,NaN,Город Краснодар
3,2024-06-08 11:43:08,176652,Новые производства и развитие промпарков: итог...,3348.0,4,0.0,post,NaN,NaN,NaN,NaN,Город Краснодар
4,2024-06-08 10:20:00,176650,🍒На летних ярмарках выходного дня — богатый ур...,2114.0,9,0.0,post,NaN,NaN,NaN,NaN,Город Краснодар
...,...,...,...,...,...,...,...,...,...,...,...,...
128,2024-06-03 03:25:29,176498,"А мне вот не мешают, чем брать гнить втридорог...",NaN,2,NaN,comment,NaN,365234283.0,176483.0,None,Город Краснодар
129,2024-06-03 04:38:01,176500,Собаки торговых сетей вышли на охоту,NaN,3,NaN,comment,NaN,596232317.0,176483.0,None,Город Краснодар
130,2024-06-04 11:04:38,176534,В панораме с..ть хотели и на участкового и на...,NaN,0,NaN,comment,NaN,279903238.0,176483.0,None,Город Краснодар
131,2024-06-04 11:52:55,176536,"Татьяна, Проблема в барыгах! Где появляется ...",NaN,1,NaN,comment,NaN,279903238.0,176483.0,None,Город Краснодар


#### Запуск геолокатора со встроенным геокодером

In [3]:
result = Geocoder(osm_id = 7373058).run(osm_id = 7373058, tags = {"place": ["city", "suburb"], "admin_level": ["9"]}, date = "2024-04-22T00:00:00Z", df=df, text_column='text', group_column='group_name')

2024-06-10 00:02:35,993 SequenceTagger predicts: Dictionary with 5 tags: O, S-ADDRESS, B-ADDRESS, E-ADDRESS, I-ADDRESS


Processing category place: 100%|██████████| 1/1 [00:00<00:00,  5.98it/s]


AttributeError: 'str' object has no attribute 'item'

In [ ]:
result.columns

In [13]:
result

,date,id,text,views.count,likes.count,reposts.count,type,link,from_id,post_id,...,initial_street,Toponims,full_street_name,location_options,addr_to_geocode,only_full_street_name,Location,geometry,other_geo_obj,geo_obj_tag
0,2024-06-09 14:20:00,176668,«Работа России. Знание — сила»: 40 школ и детс...,1547.0,2,0.0,post,NaN,NaN,NaN,...,Конгрессной,улица,Конгрессная улица 153 Krasnodar Россия,['Конгрессная улица 153 Krasnodar Россия'],Конгрессная улица 153 Krasnodar Россия,Конгрессная улица,"Конгрессная улица, Прикубанский округ, Краснод...",POINT (38.97396 45.10496),NaN,street
1,2024-06-09 10:50:00,176663,Служба по контракту: социальные льготы и гаран...,4648.0,3,0.0,post,NaN,NaN,NaN,...,Ярославская,улица,Ярославская улица 8 Krasnodar Россия,['Ярославская улица 8 Krasnodar Россия'],Ярославская улица 8 Krasnodar Россия,Ярославская улица,"8, Ярославская улица, 9-й км, Прикубанский окр...",POINT (38.98319 45.07689),NaN,street
2,2024-06-08 10:20:00,176650,🍒На летних ярмарках выходного дня — богатый ур...,2093.0,9,0.0,post,NaN,NaN,NaN,...,Архитектора Ишунина,None,улица Архитектора Ишунина Krasnodar Россия,['улица Архитектора Ишунина Krasnodar Россия'],улица Архитектора Ишунина Krasnodar Россия,улица Архитектора Ишунина,"улица Архитектора Ишунина, Фестивальный, Прику...",POINT (38.95690 45.06477),NaN,street
3,2024-06-07 08:56:12,176598,📽️ В микрорайоне Гидростроителей пройдёт киноп...,5716.0,12,2.0,post,NaN,NaN,NaN,...,Гидростроителей,None,улица Гидростроителей Krasnodar Россия,['улица Гидростроителей Krasnodar Россия'],улица Гидростроителей Krasnodar Россия,улица Гидростроителей,"улица Гидростроителей, Карасунский округ, Крас...",POINT (39.08249 45.00323),NaN,street
4,2024-06-06 10:27:31,176578,🚍 В Краснодаре изменится схема движения автобу...,6134.0,5,1.0,post,NaN,NaN,NaN,...,Тургенева,улица,улица Тургенева Krasnodar Россия,['улица Тургенева Krasnodar Россия'],улица Тургенева Krasnodar Россия,улица Тургенева,"улица Тургенева, Западный округ, Краснодар, го...",POINT (38.95521 45.03897),NaN,street
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,2024-06-03 03:25:29,176498,"А мне вот не мешают, чем брать гнить втридорог...",NaN,2,NaN,comment,NaN,365234283.0,176483.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
136,2024-06-03 04:38:01,176500,Собаки торговых сетей вышли на охоту,NaN,3,NaN,comment,NaN,596232317.0,176483.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
137,2024-06-04 11:04:38,176534,В панораме с..ть хотели и на участкового и на...,NaN,0,NaN,comment,NaN,279903238.0,176483.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
138,2024-06-04 11:52:55,176536,"Татьяна, Проблема в барыгах! Где появляется ...",NaN,1,NaN,comment,NaN,279903238.0,176483.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
